In [1]:
import pymongo
connection = pymongo.MongoClient('localhost', 27017)
db = connection.tutorial

In [11]:
from bson.objectid import ObjectId

# 4.1 스키마 설계 원리
데이터베이스 스키마 설계는 데이터베이스 시스템의 기능과 데이터의 특성, 애플리케이션의 요구사항이 주어졌을 때 데이터에 대한 최적의 표현을 찾아내는 과정이다. RDBMS는 이러한 것이 잘 확립되어 있지만, 몽고디비는 스키마 설계 규칙이 없다. 몽고디비는 '이론에 따라 스키마를 설계해야 하지만, 실제에서는 이론을 융통성 있게 적용해야한다'라는 전제를 바탕으로 한다.
- 애플리케이션 액세스 패턴은 무엇인가?
    - 애플리케이션의 요구사항이 무엇인지 정확히 파악할 필요가 있고, 이는 스키마 설계뿐만 아니라 내가 선택한 데이터베이스에도 주지되어야 하는 사항이다. 액세스 패턴을 이해하는 것은 매우 중요하다.
- 데이터베이스에는 어떤 기능이 있는가?
    - 몽고디비에는 애드혹 쿼리는 허용하지만 조인은 허용하지 않는다.그리고 기존 방식의 트랜잭션을 지원하지 않지만, 복잡한 도큐먼트의 내부 구조에 대해 수행할 수 있는 원자적인 업데이트를 다양하게 지원한다. 
- 좋은 고유 식별자와 프라이머리 키를 무엇으로 만드는가?
    - 데이터베이스에 관계없이 많은 스키마들이 고유한 키를 갖는다. 데이터를 접근하는 방식과 저장하는 방식에서 큰 차이를 만들어 낼 수 있다. 몽고디비에서 프라이머리 키를 선택하는 것은 어떤 값이 _id 필드에 채워질 것인지 고르는 것을 뜻한다. 

# 4.2 전자상거래 데이터 모델 설계
보통 RDBMS에서 강력한 모습을 보여준 데이터 모델이지만 요즘은 NoSQL에서 더 강력한 모습을 보여주고 있다. 

In [5]:
db.products.insert({'slug':'wheelbarrow-9092','sku':'9092','name':'Extra Large Wheelbarrow',
                    'description':'Heavy duty wheelbarrow...', 'details':{'weight' : 47, 
                                                                         'weight_units':'lbs',
                                                                        'model_num':4039283402,
                                                                         'manufacturer':'Acme',
                                                                         'color':'Green'},
                   'total_review':4,
                   'average_review':4.5,
                   'pricing':{'retai':589700,'sale':489700},
                   'price_history': [{
                       'retail':529700,
                       'sale':429700,
                       'start':'new Date(2010,4,1)',
                       'end':'new Date(2010, 4,8)',
                   },
                       {
                       'retail':529700,
                       'sale':529700,
                       'start':'new Date(2010,4,9)',
                       'end':'new Date(2010, 4,16)',
                   },
                       
                   ]})

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ObjectId('5ec6aee97da6a68c18330670')

In [12]:
db.products.update_one({'sku':'9092'}, {'$set':{'primary_category': ObjectId('5ebeb37d7a5d1c566f7834be')}})

In [14]:
db.products.update_one({'sku':'9092'}, {'$set':{'category_ids': [ObjectId('5ec6aee97da6a68c18330670'),
                                                                ObjectId('5ec6aee97da6a68c18330671')]}})

In [17]:
db.products.update_one({'sku':'9092'}, {'$set':{'main_cat_id': ObjectId('5ec6aee97da6a68c18330670'),
                                               'tags':['tools','gardening','soil']}})

In [22]:
# 전자 상거래 데이터
db.products.find_one()

{'_id': ObjectId('5ec6aee97da6a68c18330670'),
 'slug': 'wheelbarrow-9092',
 'sku': '9092',
 'name': 'Extra Large Wheelbarrow',
 'description': 'Heavy duty wheelbarrow...',
 'details': {'weight': 47,
  'weight_units': 'lbs',
  'model_num': 4039283402,
  'manufacturer': 'Acme',
  'color': 'Green'},
 'total_review': 4,
 'average_review': 4.5,
 'pricing': {'retai': 589700, 'sale': 489700},
 'price_history': [{'retail': 529700,
   'sale': 429700,
   'start': 'new Date(2010,4,1)',
   'end': 'new Date(2010, 4,8)'},
  {'retail': 529700,
   'sale': 529700,
   'start': 'new Date(2010,4,9)',
   'end': 'new Date(2010, 4,16)'}],
 'primary_category': ObjectId('5ebeb37d7a5d1c566f7834be'),
 'category_ids': [ObjectId('5ec6aee97da6a68c18330670'),
  ObjectId('5ec6aee97da6a68c18330671')],
 'main_cat_id': ObjectId('5ec6aee97da6a68c18330670'),
 'tags': ['tools', 'gardening', 'soil']}

전자 상거래 데이터에는 슬러그 필드를 만드는 것이 좋다. 고유 인덱스를 만들어서 해당 필드의 값이 빠른 쿼리 접근성을 갖는 동시에 고유성을 보장하도록 할 수 있다. 또한 슬러그를 _id 필드로 저장해서 프라이머리 키로 쓸 수 있다. 

In [ ]:
# Primary key를 이런식으로 바꿔주기
db.products.createIndex({'slug':1},{'unique':'true'})

- 중첩 도큐먼트는 details를 보면 알 수 있다. 여러가지 상품에 대한 자세한 정보를 갖는 서브 도큐먼트를 가르킨다. 동적인 속성을 갖기 적합하다. 
- price history와 primary_category는 일대다 관계이다. 
- category_ids는 다대다 관계이다. 몽고디비는 다대다 관계를 지원해주지 않기 때문에 category_ids라는 필드로 정의한 것이다. 

카테고리는 항상 계층 구조로 인식된다. 몽고디비는 조인을 지원하지 않기 때문에 각 자식 도큐먼트에서 조상 카테고리의 이름을 모두 가지고 있을 수 있다. 

-- 이후 생략 -- 

# 4.3 실제적 세부사항: 데이터베이스, 컬렉션, 도큐먼트

데이터베이스는 컬렉션과 인덱스의 물리적인 모음이며, 동시에 네임스페이스다. 
<br>
컬렉션은 구조적으로 혹은 개념적으로 비슷한 도큐먼트를 담고 있는 컨테이너다. 컬렉션은 별도의 명령 없이 도큐먼트를 네임스페이스에 삽입하는 것만으로도 컬렉션이 생성된다. 크기를 미리 할당해줄 수 있다.

In [ ]:
# 여기서는 에러가 난다.. 
db.createCollection('users',{'size': 20000})

캡드 컬렉션은 높은 성능의 로깅 기능을 위해 설계되었다. 일반 컬렉션과 다른 점은 고정된 크기를 가지고 있는 것이다. 캡드 컬렉션이 더 이상의 공간이 없게 되면 도큐먼트를 삽입할 때 컬렉션에 추가된 지 가장 오래된 도큐먼트를 덮어쓰게 된다. 

In [ ]:
# 여기서는 에러가 난다.. 
db.createCollection('users',{'capped':'true','max':100,'size': 20000})

TTL 컬렉션은 특정 시간이 경과한 도큐먼트를 만료시킬 수 있는 기능을 가지고 있는 컬렉션이다.
<br>
시스템 컬렉션은 내부에서 컬렉션을 부분적으로 사용하는 방법이다.   

모든 도큐먼트는 MongoDB에 저장하기 전에 BSON으로 시리얼라이즈 되고, 나중에 BSON으로 디시리얼라이즈 된다. 프로그래밍 언어를 통해 적절한 데이터 타입으로 변환된다. BSon을 디시리얼라이즈 하는 것은 StringIO 클래스를 통해 간단하게 처리할 수 있다.
키 이름이 유효해야하고 키의 값이 BSON 타입으로 변환되어야 시리얼라이즈 할 수 있다.

### BSON 타입 예제
- 문자열: 무조건 UTF-8
- 숫자: double, int, long 지원, decimal은 지원하지 않는다.
- 날짜와 시간: 시간이나 날짜에 관련된 값을 저장하는 데 사용된다. new Date를 사용한다.
- 가상 타입: 임의로 데이터 타입을 만드는 것은 불가능하지만, 여러가지 Bson 타입을 사용해서 가상의 타입을 만들 수 있다. 
